In [1]:
(define-syntax point
  (syntax-rules ()
    ( (point (a b))
         (format #f "~A ~A" a b))
    )
  )

In [2]:
(define-syntax bezier
  (syntax-rules ()
     ( (bezier sPT ePT cPT)
         `( path (d . (( ,(point sPT) ,(point ePT) ,(string-append "Q " (point cPT)) )) )
                 (stroke . "black")
                 (fill . "none")
                 (stroke-width . "2")
                 (transform . ((translate . "") (scale . "") (rotate . "") )) )
     )
  )
)

In [3]:
(define-syntax form
  (syntax-rules ()
    ((form shape ...) ; lista em define-syntax
     (let* ((get_d (lambda (curve) (cadr (assq 'd (cdr curve) ))))
            (dList (map get_d (list shape ...)))
            (sPT (caar dList) )
            (ePT (cadr (car (list-tail dList (- (length dList) 1))) ))
            (bodyList (apply append (map (lambda (path) (reverse (cdr path))) dList)) ) ; get the second and third element of every list
            (body (string-join (reverse (cdr (reverse bodyList))) " ") ) ; remove the last elemet and concatenate strings
            )
       
            `( path (d . ((,sPT ,ePT ,body)) )
                 (stroke . "black")
                 (fill . "none")
                 (stroke-width . "2")
                 (transform . ((translate . "") (scale . "") (rotate . "")) )) 
       )
     )
    )
  )

In [4]:
(define-syntax union
  (syntax-rules ()
    ((union shape ...)
         (let* ((get_shape (lambda (x)
                              (let ((type (car x))
                                    (rest (cdr x)))
                                (cond
                                  ((eq? 'group type) (car rest) )
                                  (else (cons type rest) ))))
                            )
                (temp_s (map get_shape (list shape ...)) )
                (unite (lambda (x)
                             (if (and (list? x) (list? (car x)) (eq? 'path (caar temp_s)))
                                 x
                                 (list x))))
                (shapes (apply append (map unite temp_s) ) ))
                  `( group (,@shapes))
         )
      )
  )
)

In [5]:
 (define-syntax fill
  (syntax-rules ()
    ( (fill color shape)
         (let ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (cdr (assq 'stroke (cdr shape))) )
               (newColor (format #f "~A" (quote color) ) )
               (newStrokeW (cdr (assq 'stroke-width (cdr shape))) )
               (newTransform (cdr (assq 'transform (cdr shape))) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ,newTransform))
           )
    )
  )
)

In [6]:
 (define-syntax stroke
  (syntax-rules ()
    ( (stroke color shape)
         (let ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (format #f "~A" (quote color) ) )
               (newColor  (cdr (assq 'fill (cdr shape))))
               (newStrokeW (cdr (assq 'stroke-width (cdr shape))) )
               (newTransform (cdr (assq 'transform (cdr shape))) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ,newTransform))
           )
    )
  )
)

In [7]:
 (define-syntax stroke-width
  (syntax-rules ()
    ( (stroke width shape)
         (let ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (cdr (assq 'stroke (cdr shape))) )
               (newColor  (cdr (assq 'fill (cdr shape))))
               (newStrokeW (format #f "~A" (quote width) ) )
               (newTransform (cdr (assq 'transform (cdr shape))) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ,newTransform))
           )
    )
  )
)

In [8]:
 (define-syntax translate
  (syntax-rules ()
    ( (translate pt shape)
         (let* ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (cdr (assq 'stroke (cdr shape))) )
               (newColor  (cdr (assq 'fill (cdr shape))))
               (newStrokeW (cdr (assq 'stroke-width (cdr shape))) )
               (newTransform (cdr (assq 'transform (cdr shape))) )
               (newTranslate (format #f "translate(~A)" (point pt) ) ) 
               (newScale (cdr (assq 'scale newTransform)) )
               (newRotate (cdr (assq 'rotate newTransform)) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ((translate . ,newTranslate) (scale . ,newScale) (rotate . ,newRotate) ) ))
           )
    )
  )
)

In [9]:
 (define-syntax scale
  (syntax-rules ()
    ( (scale xScale yScale shape)
         (let* ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (cdr (assq 'stroke (cdr shape))) )
               (newColor  (cdr (assq 'fill (cdr shape))))
               (newStrokeW (cdr (assq 'stroke-width (cdr shape))) )
               (newTransform (cdr (assq 'transform (cdr shape))) )
               (newTranslate (cdr (assq 'translate newTransform)) ) 
               (newScale (format #f "scale(~A ~A)" xScale yScale ) )
               (newRotate (cdr (assq 'rotate newTransform)) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ((translate . ,newTranslate) (scale . ,newScale) (rotate . ,newRotate) ) ))
           )
    )
  )
)

In [10]:
 (define-syntax rotate
  (syntax-rules ()
    ( (rotate degree shape)
         (let* ((newD (cdr (assq 'd (cdr shape))) )
               (newStroke (cdr (assq 'stroke (cdr shape))) )
               (newColor  (cdr (assq 'fill (cdr shape))))
               (newStrokeW (cdr (assq 'stroke-width (cdr shape))) )
               (newTransform (cdr (assq 'transform (cdr shape))) )
               (newTranslate (cdr (assq 'translate newTransform)) ) 
               (newScale (cdr (assq 'scale newTransform)) )
               (newRotate (format #f "rotate(~A)" degree ) ))
           
               `( path (d . ,newD )
                     (stroke . ,newStroke)
                     (fill . ,newColor)
                     (stroke-width . ,newStrokeW)
                     (transform . ((translate . ,newTranslate) (scale . ,newScale) (rotate . ,newRotate) ) ))
           )
    )
  )
)

In [11]:
(define-syntax new-panel
     (syntax-rules ()
          ( (new-panel w h)
            ( let ((width w)
                   (height h))

                (lambda (shape) 
                      (let* ((type (car shape) )
                            (content (cdr shape))
                            (toSVG (lambda (cont)
                                            (let* ((newD (cadr (assq 'd cont)) )
                                                   (sPT (car newD))
                                                   (ePT (cadr newD))
                                                   (otherD (caddr newD))
                                                   (newStroke (cdr (assq 'stroke cont)) )
                                                   (newColor (cdr (assq 'fill cont)) )
                                                   (close-str (if (string=? sPT ePT) " Z" ""))
                                                   (newStrokeW (cdr (assq 'stroke-width cont)) )
                                                   (newTransform (string-join (map cdr (cdr (assq 'transform cont ) ) ) " ")) )
                                            
                                                  `(<path
                                                          d= ,(string-append "M "
                                                                   sPT
                                                                   " "
                                                                   otherD " "
                                                                   ePT
                                                                   close-str )
                                                          stroke= ,(format #f "~A" newStroke)
                                                          fill= ,(format #f "~A" newColor)
                                                          stroke-width= ,(format #f "~A" newStrokeW)
                                                          transform= ,(format #f "~A" newTransform)
                                                        />)
                                            )

                                    ) ))

                            (cond
                                 ((eq? 'path type)
                                    
                                    (append 
                                        `(<svg
                                            width="400"
                                            height="400"
                                            viewBox= ,(format #f "0 0 ~A ~A" width height )
                                          >)
                                        (toSVG content)
                                        '(</svg>)
                                    )

                                  )
                                 (else  
                                      (append 
                                          `(<svg
                                              width="400"
                                              height="400"
                                              viewBox= ,(format #f "0 0 ~A ~A" width height )
                                            >)
                                          (map (lambda (x) (toSVG (cdr x)) ) (car content))
                                          '(</svg>)
                                      )
                                 )

                                 )
                        )

                  )
            )
          )
     )
)

In [12]:
(define-syntax defineSVG
  (syntax-rules ()
    ((defineSVG name (args ...)
       template)
     (define-syntax name
       (syntax-rules ()
         ((name args ...) template))))))